In [1]:
#quick script to find the autocorrelation in time of front frequency for each 1 degree latitude bin
#without accounting for autocorrelation, changes in frequency will be found to be more significant
#than they actually are

#a bunch of this code is copied over from RDA_hov_periods.nc and slightly adapted.

In [1]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
from autocorr import autocorr
%matplotlib notebook

In [2]:
#Access NetCDF files (currently hosted in Ferret directory)
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"
RDA_1 = nc.Dataset(RDA_path_1, 'r')
RDA_2 = nc.Dataset(RDA_path_2, 'r')

In [3]:
#load data from NetCDF files to notebook
lat_1_all =  RDA_1.variables['lat_115'][:]
lat_2_all =  RDA_2.variables['lat_115'][:]
intensity_1_all = RDA_1.variables['intensity'][:] 
intensity_2_all = RDA_2.variables['intensity'][:]

RDA_1.close()
RDA_2.close()

In [4]:
#Assign a calendar date to each time point
startday = datetime.datetime(1951,1,1)
date_list = np.array([datetime.timedelta(days=x) + startday for x in range(0, 20819)])

In [5]:
#quick function that bins everything in a 2D array and puts floor and ceiling on array values.
def findbins(lat):
    index = np.ndarray.round(lat-19.5)
    index[index < 0] = 0
    index[index > 21] = 21
    return index

In [6]:
#bin latitudes
lat_1_indices = findbins(lat_1_all)
lat_2_indices = findbins(lat_2_all)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater


In [7]:
#create a time series of whether a rainband exists for each latitude bin
occupancy_1 = np.zeros([20819,22])
occupancy_2 = np.zeros([20819,22])

for i, index in enumerate(lat_1_indices):
    if ~np.isnan(index):
        occupancy_1[i,index] = 1
        
for i, index in enumerate(lat_2_indices):
    if ~np.isnan(index):
        occupancy_2[i,index] = 1

#also create combined occupancy matrix of both primary and secondary fronts
occupancy = occupancy_1 + occupancy_2

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [12]:
print(np.mean(occupancy, axis=0))

[ 0.00048033  0.00086459  0.00384264  0.02877179  0.02656227  0.03227821
  0.03443969  0.03818627  0.04303761  0.03669725  0.03247034  0.02987656
  0.01902109  0.01998175  0.01565877  0.01508238  0.01196023  0.00979874
  0.00667659  0.00614823  0.00403478  0.00134493]


In [27]:
autocorr(occupancy[:,13])

(array([ 1.        ,  0.12924044,  0.04338737,  0.04093527,  0.0286696 ,
         0.02621655,  0.0213097 ,  0.00904133,  0.02376359,  0.00167848,
         0.00413162]), 1.6567479190260954)

In [9]:
#Define empty variables reshaped with axis 0 as day of year and axis 1 as year.
lat_1 = np.zeros([365, 57])
lat_2 = np.zeros([365, 57])
int_1 = np.zeros([365, 57])
int_2 = np.zeros([365, 57])
        
for i, date in enumerate(date_list[0:365]): #go through each day of the year

    mymap = map(lambda date: (date.day == dd) & (date.month == mm), date_list)
    extract = np.array(list(mymap))

    lat_1[i,:] = lat_1_all[extract]
    lat_2[i,:] = lat_2_all[extract]
    int_1[i,:] = intensity_1_all[extract]
    int_2[i,:] = intensity_2_all[extract]